In [1]:
import torch
import torch.autograd as autograd
import torch.nn as nn

from torchvision import models

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')
BATCH_SIZE = 32

In [3]:
model = models.vgg11(pretrained=True).to(device)
model.eval();

for layer in model.parameters():
    layer.requires_grad = False

In [4]:
layer = model.features[:6]

img_batch = torch.randn(BATCH_SIZE, 3, 224, 224, device=device)

In [5]:
#v1 = 5 * torch.ones(img_batch_copy.shape[1:])
v1 = torch.randn(150528, device=device)
#v2 = torch.ones(output_batch.shape[1:], requires_grad=True)
v2 = torch.randn(401408, requires_grad=True, device=device)

In [6]:
new_v1 = v1.clone()
new_v2 = v2.detach().clone().requires_grad_(True)

In [7]:
batch_size = img_batch.shape[0]
img_batch_copy = img_batch.view(batch_size, -1).requires_grad_(True)

if img_batch_copy.dim() == 2:
    output_batch = layer(img_batch_copy.reshape(batch_size, 3, 224, 224)).reshape(batch_size, -1)
    
#v1 = 5 * torch.ones(img_batch_copy.shape[1:])
#v2 = torch.ones(output_batch.shape[1:], requires_grad=True)

In [8]:
%%time
prod = output_batch @ v2

grad = []
for i in range(BATCH_SIZE):
    print(i)
    grad.append(autograd.grad(prod[i], img_batch_copy, create_graph=True)[0][i])


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
CPU times: user 58 s, sys: 18.5 s, total: 1min 16s
Wall time: 12.9 s


In [9]:
%%time
g = [grad[i] @ v1 for i in range(32)]
v2.data.zero_();

CPU times: user 8.55 ms, sys: 1.09 ms, total: 9.64 ms
Wall time: 2 ms


tensor([0., 0., 0.,  ..., 0., 0., 0.])

In [10]:
%%time
common_mv = [autograd.grad(g[i], v2, retain_graph=True)[0] for i in range(32)]

CPU times: user 3min 36s, sys: 45.4 s, total: 4min 21s
Wall time: 44 s


In [8]:
%%time
new_prod = output_batch @ new_v2

new_grad = autograd.grad(new_prod.sum(), img_batch_copy, create_graph=True)[0]


RuntimeError: cublas runtime error : an invalid numeric value was used as an argument at /opt/conda/conda-bld/pytorch_1565272279342/work/aten/src/THC/THCBlas.cu:120

In [8]:
%%time
new_prod = output_batch @ new_v2

new_new_grad = autograd.grad(
    new_prod,
    img_batch_copy,
    grad_outputs=torch.ones_like(new_prod, device=device),
    create_graph=True
)[0]

CPU times: user 10.4 ms, sys: 13.7 ms, total: 24.1 ms
Wall time: 23.1 ms


In [10]:
new_new_grad

tensor([[ 0.0466, -1.1997, -5.1168,  ..., -0.2571, -1.9847, -3.0508],
        [ 2.1679, -0.0437, -5.8366,  ...,  1.0069,  1.4890, -1.1506],
        [ 8.5470, -1.6165, -3.5893,  ..., -2.0583,  1.3400,  0.3777],
        ...,
        [-0.4579, -6.0328, -4.5872,  ...,  3.1793,  1.0728, -0.0277],
        [-1.0957, -0.2958, -1.4569,  ...,  1.9615, -1.5942,  1.6630],
        [-3.0009,  2.6455,  2.0456,  ...,  2.0641, -0.2099, -0.0704]],
       device='cuda:0', grad_fn=<AsStridedBackward>)

In [9]:
%%time
new_g = new_new_grad @ new_v1
#new_v2.data.zero_();

CPU times: user 301 µs, sys: 72 µs, total: 373 µs
Wall time: 224 µs


In [16]:
%%time
new_common_mv = [
    autograd.grad(new_g[i], new_v2, retain_graph=True)[0]
    for i in range(32)
]

CPU times: user 2.71 s, sys: 1.24 s, total: 3.96 s
Wall time: 3.95 s


In [17]:
new_common_mv[:5]

[tensor([0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 9.5443], device='cuda:0'),
 tensor([0., 0., 0.,  ..., 0., 0., 0.], device='cuda:0'),
 tensor([ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000, -5.2978],
        device='cuda:0'),
 tensor([0., 0., 0.,  ..., 0., 0., 0.], device='cuda:0'),
 tensor([0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 8.1710], device='cuda:0')]

In [41]:
%%time
a = autograd.grad(
    iter(new_g), [new_v2 for _ in range(16)],
    iter(torch.ones(32, 32, device=device)),
    retain_graph=True
)
print(a[:5])
print(len(a), a[0].shape)

(tensor([  41.2515, -309.2067,   66.7329,  ...,   79.7467,    0.0000,
        1347.6770], device='cuda:0'), tensor([  41.2515, -309.2067,   66.7329,  ...,   79.7467,    0.0000,
        1347.6770], device='cuda:0'), tensor([  41.2515, -309.2067,   66.7329,  ...,   79.7467,    0.0000,
        1347.6770], device='cuda:0'), tensor([  41.2515, -309.2067,   66.7329,  ...,   79.7467,    0.0000,
        1347.6770], device='cuda:0'), tensor([  41.2515, -309.2067,   66.7329,  ...,   79.7467,    0.0000,
        1347.6770], device='cuda:0'))
16 torch.Size([401408])
CPU times: user 455 ms, sys: 151 ms, total: 606 ms
Wall time: 605 ms


In [71]:
in_v2 = torch.tensor([1., 2., 1.], requires_grad=True, device=device)
tmp = torch.tensor([[0., 1., 1.], [1., 0., 1.]], device=device)
out_g = tmp @ in_v2

In [72]:
a = autograd.grad(
    out_g, [in_v2 for _ in range(2)],
    torch.eye(2, device=device),
    retain_graph=True
)
print(a)

(tensor([1., 1., 2.], device='cuda:0'), tensor([1., 1., 2.], device='cuda:0'))


In [76]:
a = autograd.grad(
    out_g, [in_v2 for _ in range(2)],
    [torch.tensor([1.0, 0.], device=device), torch.tensor([0., 1.], device=device)],
    retain_graph=True
)
print(a)

(tensor([0., 1., 1.], device='cuda:0'), tensor([0., 1., 1.], device='cuda:0'))


In [89]:
a = autograd.grad(
    iter(out_g), [in_v2 for _ in range(2)],
    iter(torch.tensor([[1.0, 1.0], [1., 0.], [56.0, 12.0]], device=device)),
    retain_graph=True
)
print(a)

(tensor([1., 2., 3.], device='cuda:0'), tensor([1., 2., 3.], device='cuda:0'))


In [46]:
%%time
new_new_common_mv = autograd.grad(
    new_g,
    [new_v2 for _ in range(BATCH_SIZE)],
    grad_outputs=torch.eye(32, device=device),
    retain_graph=True
)
new_new_common_mv.shape

AttributeError: 'tuple' object has no attribute 'shape'

In [47]:
new_new_common_mv

(tensor([  1.9591,   1.0481,  -4.4336,  ...,   0.0000,   0.0000, -31.6963],
        device='cuda:0'),
 tensor([  1.9591,   1.0481,  -4.4336,  ...,   0.0000,   0.0000, -31.6963],
        device='cuda:0'),
 tensor([  1.9591,   1.0481,  -4.4336,  ...,   0.0000,   0.0000, -31.6963],
        device='cuda:0'),
 tensor([  1.9591,   1.0481,  -4.4336,  ...,   0.0000,   0.0000, -31.6963],
        device='cuda:0'),
 tensor([  1.9591,   1.0481,  -4.4336,  ...,   0.0000,   0.0000, -31.6963],
        device='cuda:0'),
 tensor([  1.9591,   1.0481,  -4.4336,  ...,   0.0000,   0.0000, -31.6963],
        device='cuda:0'),
 tensor([  1.9591,   1.0481,  -4.4336,  ...,   0.0000,   0.0000, -31.6963],
        device='cuda:0'),
 tensor([  1.9591,   1.0481,  -4.4336,  ...,   0.0000,   0.0000, -31.6963],
        device='cuda:0'),
 tensor([  1.9591,   1.0481,  -4.4336,  ...,   0.0000,   0.0000, -31.6963],
        device='cuda:0'),
 tensor([  1.9591,   1.0481,  -4.4336,  ...,   0.0000,   0.0000, -31.6963],
      

In [21]:
autograd.grad()

torch.Size([32])

In [22]:
new_v2.shape

torch.Size([401408])

In [29]:
lel = autograd.grad(
    output_batch, 
    img_batch_copy,
    grad_outputs=[torch.ones(32, 401408), torch.ones(32, 401408), v1], 
    retain_graph=True
)[0]


RuntimeError: invalid gradient at index 0 - expected type torch.cuda.FloatTensor but got torch.FloatTensor

In [13]:
pupa = torch.stack(common_mv)
lupa = torch.stack(new_common_mv)

In [13]:
id(img_batch.contiguous())

140454705383896

In [14]:
id(img_batch)

140454705383896

In [1]:
import torch
import torch.autograd as autograd
import torch.nn as nn

In [13]:
layer = nn.Sequential(
    nn.Linear(1000, 500),
    nn.ReLU()
)

img_batch = torch.randn(32, 1000)

In [14]:
#v1 = 5 * torch.ones(img_batch_copy.shape[1:])
v1 = torch.randn(1000)
#v2 = torch.ones(output_batch.shape[1:], requires_grad=True)
v2 = torch.randn(500, requires_grad=True)

In [4]:
v2[:100]

tensor([ 0.2546,  0.2825,  0.2804,  1.0867,  0.7721,  0.3622, -0.5979, -0.5619,
        -0.8070,  0.4958,  0.0262, -0.4871, -1.2632,  1.1464,  0.0784,  0.2070,
        -0.2882, -1.5814, -0.5747,  0.4433,  0.8672,  1.2101,  0.9946, -2.5411,
         1.3365, -0.6848, -0.3588,  0.1659,  0.6818, -0.1192, -0.0901,  1.2334,
         2.8858, -1.4674, -2.0275,  0.6004, -1.4563,  0.0986, -0.1476, -0.2667,
        -0.3497, -0.6536, -0.5182,  0.8500, -0.1064, -1.7516, -0.4474,  0.0311,
         0.7674,  1.1715, -0.5634,  0.3112,  0.7803, -0.2004,  1.1978, -0.4738,
        -0.2542, -1.3166,  0.0118, -2.2514,  0.6150, -2.2872,  0.3073,  0.9277,
        -0.4295, -1.8750,  0.6777,  0.4519,  1.1443,  1.0001,  0.8119, -0.3817,
         0.0789, -0.9182, -1.6049, -0.4204,  0.4300, -0.0655,  0.2086, -0.0031,
         0.0825,  0.1779,  0.9970,  0.0059,  0.6481,  0.5044, -0.6111, -2.1053,
         1.3982, -1.0493, -0.4526, -0.9328, -0.9673, -0.6060, -2.0955, -0.5374,
        -0.9446, -1.2989,  0.3348,  0.31

In [15]:
img_batch_copy = img_batch.clone().requires_grad_(True)

output_batch = layer(img_batch_copy)
    
#v1 = 5 * torch.ones(img_batch_copy.shape[1:])
#v2 = torch.ones(output_batch.shape[1:], requires_grad=True)

In [16]:
output_batch

tensor([[0.1110, 0.0000, 0.0000,  ..., 0.0000, 0.5476, 0.8789],
        [0.6741, 0.0000, 0.0000,  ..., 0.0000, 0.4007, 0.0521],
        [0.0000, 0.0000, 0.6442,  ..., 0.8388, 0.0000, 0.5958],
        ...,
        [0.8039, 0.1303, 0.0000,  ..., 0.0000, 0.4765, 0.0000],
        [0.2398, 0.0037, 0.3700,  ..., 0.0402, 0.6447, 1.4258],
        [0.0000, 0.6830, 0.0000,  ..., 0.4809, 0.0000, 0.7615]],
       grad_fn=<ReluBackward0>)

In [17]:
prod = output_batch @ v2

grad = []
for i in range(32):
    grad.append(autograd.grad(prod[i], img_batch_copy, retain_graph=True, create_graph=True)[0][i])



In [18]:
grad[0][:100]

tensor([ 1.6601e-01, -3.6088e-01, -1.9948e-02,  4.3331e-01,  2.0584e-02,
         3.7855e-02,  6.9233e-01,  3.2370e-01, -3.0797e-01, -1.9091e-01,
         2.7293e-01, -1.1951e-01, -7.6927e-02, -9.8618e-02,  1.6786e-01,
        -3.2780e-01, -3.7215e-01,  1.6748e-01,  2.7731e-01, -4.1819e-02,
        -3.6323e-01, -2.1436e-01,  2.7398e-01,  4.4455e-01,  1.0986e-01,
         2.9508e-02,  9.2924e-01,  3.1152e-01,  2.1843e-01,  7.6213e-02,
        -1.3701e-01,  2.5774e-02,  6.8287e-02, -8.6855e-01,  2.8958e-01,
        -1.8949e-01, -9.8556e-02,  2.9287e-01,  8.2704e-03,  3.2341e-01,
        -1.8028e-01,  9.8288e-02,  1.2090e-01, -2.1645e-01, -5.2486e-02,
        -2.3969e-02, -7.0296e-02, -9.8526e-02, -1.7235e-01,  4.3142e-01,
         5.8110e-01,  5.0458e-01, -1.0733e-01,  1.6641e-01, -2.3055e-01,
         3.9658e-01,  5.7112e-02, -4.1066e-01,  1.9311e-01, -1.7012e-01,
         1.1305e-01, -4.5183e-01,  1.2717e-04, -6.3801e-02,  4.0950e-01,
        -2.4851e-01, -1.0898e-01,  1.4688e-01,  3.0

In [19]:
grad[1][:100]

tensor([ 1.0522e-01, -4.3441e-01,  6.8575e-02,  1.9317e-01,  2.3089e-01,
         8.9383e-02,  4.8135e-01,  3.4231e-01, -2.8888e-02,  2.9547e-01,
         2.8443e-01, -2.8834e-01, -3.3908e-01, -6.9788e-02, -2.1918e-01,
        -8.6357e-02, -3.1047e-01, -2.9438e-01,  3.0658e-01, -2.4802e-01,
         1.2108e-01, -6.8747e-02,  2.7700e-01,  7.4928e-01, -1.6137e-01,
         1.9986e-01,  8.6020e-01, -3.2498e-03,  3.9949e-01, -6.5910e-03,
         3.8083e-01,  9.6820e-02, -1.7064e-01, -4.6541e-01,  3.8144e-01,
         4.1289e-01, -4.5711e-01,  1.2691e-01,  3.8861e-01,  2.4503e-01,
         1.4239e-01,  1.1239e-01, -2.2420e-01, -1.4887e-01, -2.0484e-01,
         8.3919e-04,  3.8844e-01, -1.0954e-01,  2.6535e-02,  1.7529e-01,
         2.3041e-01,  3.5464e-01,  3.1174e-01, -1.2376e-01,  7.1551e-02,
         1.5415e-01, -1.3680e-02, -2.2070e-01, -3.1524e-01, -1.0761e-01,
         3.6819e-01, -3.7579e-01,  1.5527e-02, -8.0767e-02,  5.8959e-01,
        -4.3346e-02, -3.5617e-01,  1.6644e-01,  8.3

In [11]:
v2[:100]

tensor([ 0.2546,  0.2825,  0.2804,  1.0867,  0.7721,  0.3622, -0.5979, -0.5619,
        -0.8070,  0.4958,  0.0262, -0.4871, -1.2632,  1.1464,  0.0784,  0.2070,
        -0.2882, -1.5814, -0.5747,  0.4433,  0.8672,  1.2101,  0.9946, -2.5411,
         1.3365, -0.6848, -0.3588,  0.1659,  0.6818, -0.1192, -0.0901,  1.2334,
         2.8858, -1.4674, -2.0275,  0.6004, -1.4563,  0.0986, -0.1476, -0.2667,
        -0.3497, -0.6536, -0.5182,  0.8500, -0.1064, -1.7516, -0.4474,  0.0311,
         0.7674,  1.1715, -0.5634,  0.3112,  0.7803, -0.2004,  1.1978, -0.4738,
        -0.2542, -1.3166,  0.0118, -2.2514,  0.6150, -2.2872,  0.3073,  0.9277,
        -0.4295, -1.8750,  0.6777,  0.4519,  1.1443,  1.0001,  0.8119, -0.3817,
         0.0789, -0.9182, -1.6049, -0.4204,  0.4300, -0.0655,  0.2086, -0.0031,
         0.0825,  0.1779,  0.9970,  0.0059,  0.6481,  0.5044, -0.6111, -2.1053,
         1.3982, -1.0493, -0.4526, -0.9328, -0.9673, -0.6060, -2.0955, -0.5374,
        -0.9446, -1.2989,  0.3348,  0.31

In [25]:
img_copy = img_batch[0].clone().requires_grad_(True)
output = layer(img_copy).squeeze(0)

In [26]:
J_T_v2 = autograd.grad(v2.T @ output, img_copy, create_graph=True)[0].squeeze(0)

In [28]:
J_T_v2[:100]

tensor([-0.0967,  0.6613,  0.0904, -0.5911, -0.2223,  0.0473,  0.2573, -0.5294,
        -0.6959,  0.4946, -0.2523,  0.0323, -0.5638, -0.1927, -0.4454,  0.8830,
        -0.0125,  0.2515, -0.5234, -0.2306, -0.1667,  0.2894, -0.6074, -1.1030,
         0.2901, -0.0295, -0.0454,  0.2911,  0.3025, -0.1430, -0.4111,  0.0812,
        -0.2916,  0.0162,  0.3134, -1.0102, -0.1975,  0.3630, -0.6655,  0.1572,
         0.1311,  0.0599, -0.3220,  0.1571, -0.4180, -0.4647, -0.0366, -0.2610,
        -0.2052,  0.6103,  0.8097, -0.3804,  0.5247, -0.7721,  0.3280,  0.5229,
        -1.5179, -0.7838, -0.7823, -0.0840, -0.0044,  0.0189, -0.1848, -0.0462,
        -0.0748,  0.5890,  0.9152,  0.0908, -0.1914, -0.2719, -0.0801, -0.0293,
        -0.2030, -0.1667, -0.8302,  0.3030,  0.2898, -0.3585,  0.8013, -0.8045,
         0.1601, -0.3660,  0.2089,  0.0887, -0.2815, -0.1210,  0.2246,  0.0129,
        -0.1660,  0.0269,  0.1965,  0.4410,  0.4809,  0.4523,  0.2827,  0.2874,
        -0.4194, -0.2391,  0.2770, -0.92

In [29]:
img_copy = img_batch[28].clone().requires_grad_(True)
output = layer(img_copy).squeeze(0)

In [30]:
J_T_v2 = autograd.grad(v2.T @ output, img_copy, create_graph=True)[0].squeeze(0)

In [31]:
J_T_v2[:100]

tensor([-0.0967,  0.6613,  0.0904, -0.5911, -0.2223,  0.0473,  0.2573, -0.5294,
        -0.6959,  0.4946, -0.2523,  0.0323, -0.5638, -0.1927, -0.4454,  0.8830,
        -0.0125,  0.2515, -0.5234, -0.2306, -0.1667,  0.2894, -0.6074, -1.1030,
         0.2901, -0.0295, -0.0454,  0.2911,  0.3025, -0.1430, -0.4111,  0.0812,
        -0.2916,  0.0162,  0.3134, -1.0102, -0.1975,  0.3630, -0.6655,  0.1572,
         0.1311,  0.0599, -0.3220,  0.1571, -0.4180, -0.4647, -0.0366, -0.2610,
        -0.2052,  0.6103,  0.8097, -0.3804,  0.5247, -0.7721,  0.3280,  0.5229,
        -1.5179, -0.7838, -0.7823, -0.0840, -0.0044,  0.0189, -0.1848, -0.0462,
        -0.0748,  0.5890,  0.9152,  0.0908, -0.1914, -0.2719, -0.0801, -0.0293,
        -0.2030, -0.1667, -0.8302,  0.3030,  0.2898, -0.3585,  0.8013, -0.8045,
         0.1601, -0.3660,  0.2089,  0.0887, -0.2815, -0.1210,  0.2246,  0.0129,
        -0.1660,  0.0269,  0.1965,  0.4410,  0.4809,  0.4523,  0.2827,  0.2874,
        -0.4194, -0.2391,  0.2770, -0.92

In [59]:
output_batch[0].sum()

(tensor([[ 0.4045,  0.7945,  0.6925,  ..., -0.9007, -0.6260, -0.2170],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]),)

In [34]:
it = iter(img_batch_copy)

In [46]:
output_batch.shape

torch.Size([5, 14])

In [13]:
prod = output_batch @ v2

In [18]:
autograd.grad(prod[i - 1], img_batch_copy, retain_graph=True, create_graph=True)

(tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0292,  0.6687, -0.5288,  ...,  1.2759, -0.4639,  0.0093],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
        grad_fn=<AsStridedBackward>),)

In [19]:
autograd.grad(prod[i], img_batch_copy, retain_graph=True, create_graph=True)

(tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0292,  0.6687, -0.5288,  ...,  1.2759, -0.4639,  0.0093]],
        grad_fn=<AsStridedBackward>),)

In [14]:
grad = []
for i in range(32):
    grad.append(autograd.grad(prod[i], img_batch_copy, retain_graph=True, create_graph=True)[0][i])


In [15]:
grad

[tensor([ 0.0292,  0.6687, -0.5288,  ...,  1.2759, -0.4639,  0.0093],
        grad_fn=<SelectBackward>),
 tensor([ 0.0292,  0.6687, -0.5288,  ...,  1.2759, -0.4639,  0.0093],
        grad_fn=<SelectBackward>),
 tensor([ 0.0292,  0.6687, -0.5288,  ...,  1.2759, -0.4639,  0.0093],
        grad_fn=<SelectBackward>),
 tensor([ 0.0292,  0.6687, -0.5288,  ...,  1.2759, -0.4639,  0.0093],
        grad_fn=<SelectBackward>),
 tensor([ 0.0292,  0.6687, -0.5288,  ...,  1.2759, -0.4639,  0.0093],
        grad_fn=<SelectBackward>),
 tensor([ 0.0292,  0.6687, -0.5288,  ...,  1.2759, -0.4639,  0.0093],
        grad_fn=<SelectBackward>),
 tensor([ 0.0292,  0.6687, -0.5288,  ...,  1.2759, -0.4639,  0.0093],
        grad_fn=<SelectBackward>),
 tensor([ 0.0292,  0.6687, -0.5288,  ...,  1.2759, -0.4639,  0.0093],
        grad_fn=<SelectBackward>),
 tensor([ 0.0292,  0.6687, -0.5288,  ...,  1.2759, -0.4639,  0.0093],
        grad_fn=<SelectBackward>),
 tensor([ 0.0292,  0.6687, -0.5288,  ...,  1.2759, -0.4

In [20]:
img_batch_copy

tensor([[ 1.7696, -1.3836,  1.5466,  ..., -0.4911, -1.9428,  1.3286],
        [-1.7435,  0.1724,  0.4237,  ..., -0.1114,  0.7424, -1.0351],
        [-0.7715, -0.7562,  0.1746,  ..., -0.1764, -0.4345, -0.0455],
        ...,
        [ 1.1638, -0.1136, -0.0086,  ...,  0.9239,  0.9385,  0.6597],
        [ 0.8228, -0.3939, -0.7540,  ..., -1.3014, -0.2386, -0.7362],
        [ 1.5424, -0.5956,  0.4882,  ...,  1.3647,  0.3631, -0.4734]],
       requires_grad=True)

In [115]:
autograd.grad(prod[i], img_batch_copy, create_graph=True)

(tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [-0.0777, -0.6286, -0.6025,  ...,  0.4405,  0.5199,  0.1628]],
        grad_fn=<AsStridedBackward>),)

In [84]:
full_grad = autograd.grad(prod, img_batch_copy, retain_graph=True)

RuntimeError: grad can be implicitly created only for scalar outputs

In [102]:
J_T_v2 = img_batch_copy.grad

In [104]:
img_batch_copy.

torch.Size([32, 12288])

In [105]:
g = J_T_v2 @ v1

In [106]:
g

tensor([75069.2734, 75069.2734, 75069.2734, 75069.2734, 75069.2734, 75069.2734,
        75069.2734, 75069.2734, 75069.2734, 75069.2734, 75069.2734, 75069.2734,
        75069.2734, 75069.2734, 75069.2734, 75069.2734, 75069.2734, 75069.2734,
        75069.2734, 75069.2734, 75069.2734, 75069.2734, 75069.2734, 75069.2734,
        75069.2734, 75069.2734, 75069.2734, 75069.2734, 75069.2734, 75069.2734,
        75069.2734, 75069.2734], grad_fn=<MvBackward>)

In [111]:
v2.grad.zero_()
v2.data.zero_()

AttributeError: 'Tensor' object has no attribute 'zero_grad'

In [18]:
img_batch

tensor([[[[ 5.0238e-01, -4.1303e-01,  5.7906e-01,  ..., -1.3733e+00,
           -1.7838e+00,  1.8186e+00],
          [-1.1135e-01, -7.8527e-01, -4.3307e-01,  ...,  1.2721e+00,
           -3.2110e-01,  1.2528e-01],
          [-1.1080e+00, -1.1124e+00, -8.9090e-01,  ...,  1.8135e+00,
           -1.3762e+00, -3.5865e-01],
          ...,
          [-1.3235e-01,  1.9272e-01,  1.3293e+00,  ...,  1.0934e+00,
            1.0545e+00, -5.4225e-01],
          [-7.8190e-01, -1.1393e+00, -1.5905e+00,  ...,  1.5941e+00,
           -6.1499e-01,  6.3828e-02],
          [-3.6597e-01, -1.8701e+00,  1.7525e-01,  ..., -1.2583e+00,
           -1.2127e-01,  3.9368e-03]],

         [[-8.0723e-01, -3.4776e-01,  5.8503e-01,  ..., -8.9609e-01,
            7.6539e-01,  5.5952e-01],
          [-3.2787e-02,  1.6630e+00, -3.0340e-01,  ...,  5.6782e-01,
            6.1339e-01,  4.0386e-01],
          [-7.1178e-01, -1.6168e+00, -4.8105e-01,  ..., -1.9892e+00,
           -5.6242e-01,  2.8317e-01],
          ...,
     

In [20]:
torch.zeros_like(img_batch_copy).dtype

torch.float32

In [26]:
type(next(model.parameters()).device)

torch.device

In [29]:
print(next(layer.parameters()).device)

cuda:0


In [14]:
import numpy as np